# Exploring the NASDAQOMX india indices

In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

nasdaqOmx <- NasdaqOmx()

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend

Registering fonts with R

Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine


Attaching package

In [2]:
indexMeta <- nasdaqOmx$Meta() %>%
    inner_join(nasdaqOmx$TimeSeries()) %>%
    group_by(ID, CODE, NAME) %>%
    filter(NAME %like% '% India %') %>%
    summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
    arrange(NAME) %>%
    collect()

indexMeta %>%
    print(n = Inf)

Joining, by = "ID"
Warning message:
“Missing values are always removed in SQL.
Use `MIN(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# A tibble: 11 x 5
# Groups:   ID, CODE [11]
         ID CODE     NAME                                  ST         ET        
      <int> <chr>    <chr>                                 <date>     <date>    
 1 13120874 NQIN100… NASDAQ India Basic Matls TR Index (N… 2001-03-30 2019-07-09
 2 13120916 NQIN300… NASDAQ India Cnsmr Goods TR Index (N… 2001-03-30 2019-07-09
 3 13120939 NQIN500… NASDAQ India Cnsmr Svcs TR Index (NQ… 2005-03-31 2019-07-09
 4 13120986 NQIN800… NASDAQ India Financials TR Index (NQ… 2001-03-30 2019-07-09
 5 13121029 NQIN400… NASDAQ India Health Care TR Index (N… 2001-03-30 2019-07-09
 6 13121072 NQIN200… NASDAQ India Inds TR Index (NQIN2000… 2001-03-30 2019-07-09
 7 13121187 NQIN000… NASDAQ India Oil & Gas TR Index (NQI… 2004-09-30 2019-07-09
 8 13121253 NQIN900… NASDAQ India Tech TR Index (NQIN9000… 2001-03-30 2019-07-09
 9 13121274 NQIN600… NASDAQ India Telecom TR Index (NQIN6… 2006-03-31 2019-07-09
10 13121272 NQINT    NASDAQ India TR Index (NQINT)         2001-

In [3]:
#grab the data
startDate <- max(indexMeta$ST)
endDate <- min(indexMeta$ET)

nqRets <- NULL

for(id in indexMeta$ID){
    nqVals <- nasdaqOmx$TimeSeries() %>%
        filter(ID == id1 & TIME_STAMP >= startDate & TIME_STAMP <= endDate) %>%
        select(TIME_STAMP, CLOSE) %>%
        collect()
    
    nqRets <- merge.xts(nqRets, dailyReturn(nqVals$CLOSE, nqVals$TIME_STAMP))
}

names(nqRets) <- indexMeta$CODE

ERROR: Error: <SQL> 'SELECT "TIME_STAMP", "CLOSE"
FROM (SELECT TOP 100 PERCENT "ID", "TRADE_DATE" AS "TIME_STAMP", "INDEX_VALUE" AS "CLOSE"
FROM "QUANDL_DATA_V3") "dbplyr_002"
WHERE ("ID" = "id1" AND "TIME_STAMP" >= '2006-03-31' AND "TIME_STAMP" <= '2019-07-09')'
  nanodbc/nanodbc.cpp:1587: 42S22: [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid column name 'id1'. 


In [4]:
#plot cumulative returns
Common.PlotCumReturns(nqRets, "NASDAQ India Focused Indices (TR)", "in USD")

<simpleError in checkData(R): The data cannot be converted into a time series.  If you are trying to pass in names from a data object with one column, you should use the form 'data[rows, columns, drop = FALSE]'.  Rownames should have standard date formats, such as '1985-03-15'. >


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)